# Laboratorium 4 - Efekt Rungego

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.interpolate as interpolate

sns.set_style("darkgrid")

### Zadanie 1.
Napisz funkcję, która przyjmuje jako parametr wektor punktów $x_0, ..., x_n$ z przedziału $[-1,1]$ i tworzy wykres z punktami $x_j$ na osi odciętych i
średnią geometryczną odległości do pozostałych punktów na osi rzędnych. Wyświetl wyniki dla:
- punktów Czebyszewa dla $n = 10, 20, 50$
- punktów Legendre’a dla $n = 10, 20, 50$
- punktów równomiernie rozmieszczonych od $x_0 = −1$ do $x_n = 1$ dla $n = 10, 20, 50$.

In [2]:
def geometric_mean_distances(points):
    n = len(points)
    distances = np.zeros(n)
    for i in range(n):
        point_distances = np.abs(points[i] - points)
        point_distances = point_distances[point_distances != 0]
        distances[i] = np.power(np.prod(point_distances), 1 / (n - 1))
    return distances


def plot_geometric_mean_distances(points, point_type, n):
    distances = geometric_mean_distances(points)
    plt.plot(points, distances)


n_values = [10, 20, 50]


for n in n_values:
    plt.figure(figsize=(10, 6))
    chebyshev_points = -np.cos((np.arange(n) / n) * np.pi)
    plot_geometric_mean_distances(chebyshev_points, "Czebyszewa", n)
    legendre_points = np.polynomial.legendre.leggauss(n)[0]
    plot_geometric_mean_distances(legendre_points, "Legendre'a", n)
    uniform_points = np.linspace(-1, 1, n)
    plot_geometric_mean_distances(uniform_points, "równomiernie rozmieszczone", n)
    plt.title(f"Średnia geometryczna odległości dla (n={n})")
    plt.xlabel("x")
    plt.ylabel("Średnia geometryczna odległość")
    plt.legend(["Czebyszewa", "Legendre'a", "Równomiernie rozmieszczone"])
    plt.show()


<Figure size 3000x1800 with 1 Axes>

<Figure size 3000x1800 with 1 Axes>

<Figure size 3000x1800 with 1 Axes>

### Zadanie 2
Wyznacz wielomiany interpolujące funkcje

$$f_1(x) = \frac{1}{1 + 25x^2} \text{ na przedziale } [-1, 1]\,,$$

$$f_2(x) = \exp(\cos(x)) \text{ na przedziale } [0, 2\pi]\,,$$

używając:
- wielomianów Lagrange’a z równoodległymi węzłami
- kubicznych funkcji sklejanych z równoodległymi węzłami
- wielomianów Lagrange’a z węzłami Czebyszewa

$$x_j = -\cos(\theta_j) \ \ \ \theta_j = \frac{2j - 1}{2n}\pi, \; 1 \leq j \leq n\,.$$

Dla funkcji Rungego, $f_1(x)$, z $n=12$ węzłami interpolacji
przedstaw na wspólnym wykresie funkcję $f_1(x)$ oraz wyznaczone wielomiany interpolacyjne i funkcję sklejaną.
W celu stworzenia wykresu wykonaj próbkowanie funkcji $f_1(x)$ i wielomianów interpolacyjnych na 10 razy gęstszym zbiorze
(próbkowanie jednostajne w $x$ dla węzłów równoodległych, jednostajne w $\theta$ dla węzłów Czebyszewa).
Pamiętaj o podpisaniu wykresu i osi oraz o legendzie.

In [3]:
def f1(x):
    return 1 / (1 + 25 * x**2)


def f2(x):
    return np.exp(np.cos(x))


def chebyshev_nodes(n, a, b):
    j = np.arange(1, n + 1)
    theta = (2 * j - 1) * np.pi / (2 * n)
    x = 0.5 * (a + b) + 0.5 * (b - a) * np.cos(theta)
    return x


def lagrange_interpolation_equidistant(f, a, b, n, x_plot):
    x_nodes = np.linspace(a, b, n)
    y_nodes = f(x_nodes)
    poly = interpolate.BarycentricInterpolator(x_nodes, y_nodes)
    y_plot = poly(x_plot)
    return y_plot


def cubic_spline_interpolation_equidistant(f, a, b, n, x_plot):
    x_nodes = np.linspace(a, b, n)
    y_nodes = f(x_nodes)
    spline = interpolate.CubicSpline(x_nodes, y_nodes)
    y_plot = spline(x_plot)
    return y_plot


def lagrange_interpolation_chebyshev(f, a, b, n, x_plot):
    x_nodes = chebyshev_nodes(n, a, b)
    y_nodes = f(x_nodes)
    poly = interpolate.BarycentricInterpolator(x_nodes, y_nodes)
    y_plot = poly(x_plot)
    return y_plot


def plot_interpolation_runge(f, a, b, n):
    x_plot = np.linspace(a, b, 500)
    y_true = f(x_plot)

    y_lagrange_equidistant = lagrange_interpolation_equidistant(f, a, b, n, x_plot)
    y_spline_equidistant = cubic_spline_interpolation_equidistant(f, a, b, n, x_plot)
    y_lagrange_chebyshev = lagrange_interpolation_chebyshev(f, a, b, n, x_plot)

    plt.figure(figsize=(10, 6))
    plt.plot(x_plot, y_true, label="True function")
    plt.plot(x_plot, y_lagrange_equidistant, label="Lagrange (equidistant)")
    plt.plot(x_plot, y_spline_equidistant, label="Cubic spline (equidistant)")
    plt.plot(x_plot, y_lagrange_chebyshev, label="Lagrange (Chebyshev)")
    plt.legend()
    plt.title(f"Interpolacja funkcji Rungego f1(x) dla n = {n}")
    plt.xlabel("x")
    plt.ylabel("f1(x)")
    plt.grid(True)
    plt.show()


n = 12
plot_interpolation_runge(f1, -1, 1, n)

<Figure size 3000x1800 with 1 Axes>

Wykonaj interpolację funkcji $f_1(x)$ i $f_2(x)$ z $n = 4, 5, ..., 50$ węzłami interpolacji,
używając każdej z powyższych trzech metod interpolacji.
Ewaluację funkcji, wielomianów interpolacyjnych oraz funkcji sklejanych przeprowadź na
zbiorze 500 losowo wybranych punktów z dziedziny funkcji.
Stwórz dwa rysunki, jeden dla $f_1(x)$, drugi dla $f_2(x)$.
Na każdym rysunku przedstaw razem wykresy normy wektora błędów (czyli długości wektora) na
tym zbiorze punktów w zależności od liczby węzłów interpolacji, $n$, dla każdej z trzech metod interpolacji.

Która metoda interpolacji jest najbardziej dokładna, a która najmniej?

In [4]:
def calculate_error_norm(f, a, b, n_values, num_points=500):
    error_norms_lagrange_equidistant = []
    error_norms_spline_equidistant = []
    error_norms_lagrange_chebyshev = []

    for n in n_values:
        x_plot = np.random.uniform(a, b, num_points)
        y_true = f(x_plot)

        y_lagrange_equidistant = lagrange_interpolation_equidistant(f, a, b, n, x_plot)
        y_spline_equidistant = cubic_spline_interpolation_equidistant(
            f, a, b, n, x_plot
        )
        y_lagrange_chebyshev = lagrange_interpolation_chebyshev(f, a, b, n, x_plot)

        error_norms_lagrange_equidistant.append(
            np.linalg.norm(y_true - y_lagrange_equidistant)
        )
        error_norms_spline_equidistant.append(
            np.linalg.norm(y_true - y_spline_equidistant)
        )
        error_norms_lagrange_chebyshev.append(
            np.linalg.norm(y_true - y_lagrange_chebyshev)
        )

    return (
        error_norms_lagrange_equidistant,
        error_norms_spline_equidistant,
        error_norms_lagrange_chebyshev,
    )


def plot_error_norms(
    n_values,
    error_norms_lagrange_equidistant,
    error_norms_spline_equidistant,
    error_norms_lagrange_chebyshev,
    title,
):
    plt.figure(figsize=(10, 6))
    plt.semilogy(
        n_values, error_norms_lagrange_equidistant, label="Lagrange (equidistant)"
    )
    plt.semilogy(
        n_values, error_norms_spline_equidistant, label="Cubic spline (equidistant)"
    )
    plt.plot(n_values, error_norms_lagrange_chebyshev, label="Lagrange (Chebyshev)")
    plt.xlabel("Number of nodes (n)")
    plt.ylabel("Error norm")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


n_values = np.arange(4, 51)
num_points = 500


error_norms_f1 = calculate_error_norm(f1, -1, 1, n_values, num_points)
plot_error_norms(n_values, *error_norms_f1, "Error norms for f1(x)")


error_norms_f2 = calculate_error_norm(f2, 0, 2 * np.pi, n_values, num_points)
plot_error_norms(n_values, *error_norms_f2, "Error norms for f2(x)")

<Figure size 3000x1800 with 1 Axes>

<Figure size 3000x1800 with 1 Axes>

### Wnioski
Celem zadania było wykrycie efektu Rungego, polegającego na pogarszaniu się dokładności interpolacji przy zwiększaniu liczby węzłów w pewnych przypadkach, co jest kontrintuicyjne. Wykorzystane zostały do tego trzy różne metody interpolacji: wielomianów Lagrange'a z węzłami równoodległymi i Czebyszewa, i metody funkcji sklejanych.

Analiza wyraźnie wykazała, że metoda z użyciem wielomianów Lagrange'a ulega degeneracji i po zwiększeniu liczby węzłów rośnie również błąd interpolacji. Nie dzieje się tak dla metody funkcji sklejanych, która wykazała się znacznie większą dokładnością, która rośnie wraz z liczbą węzłów.

Można zauważyć, że użycie węzłów Czebyszewa sprawia, że błąd jest mniejszy i w przypadku funkcji testowanej f1 widać, że błąd maleje wraz ze wzrostem liczby węzłów, jednak trend nie jest stały i nadal występuje efekt Rungego dla większej liczby węzłów.